In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# random
from numpy.random import rand, randn

# dataframes
import pandas as pd

# plotting
import matplotlib.pyplot as plt
# linear regression two ways
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
#This is so that you don't see a bunch of code 'warnings' (things that you could change but don't have to right now)
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from sklearn import datasets, linear_model 

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount("/drive")

Mounted at /drive


In [ ]:
boa_df=pd.read_csv('/drive/My Drive/orie4741 project/BOA2018.csv')
# citi_df=pd.read_csv('/drive/My Drive/orie4741 project/CITI.csv')
# jpm_df=pd.read_csv('/drive/My Drive/orie4741 project/JPMORGAN.csv')

# **Data Frame Overview**

In [ ]:
boa_df.shape

(377468, 99)

In [ ]:
boa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377468 entries, 0 to 377467
Data columns (total 99 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   activity_year                             377468 non-null  int64  
 1   lei                                       377468 non-null  object 
 2   derived_msa-md                            377468 non-null  int64  
 3   state_code                                377468 non-null  object 
 4   county_code                               377468 non-null  int64  
 5   census_tract                              377468 non-null  int64  
 6   conforming_loan_limit                     377374 non-null  object 
 7   derived_loan_product_type                 377468 non-null  object 
 8   derived_dwelling_category                 377468 non-null  object 
 9   derived_ethnicity                         377468 non-null  object 
 10  derived_race        

In [ ]:
boa_df.describe()

,activity_year,derived_msa-md,county_code,census_tract,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open-end_line_of_credit,business_or_commercial_purpose,loan_amount,loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,multifamily_affordable_units,income,applicant_credit_score_type,co-applicant_credit_score_type,applicant_ethnicity-1,...,co-applicant_ethnicity-2,co-applicant_ethnicity-3,co-applicant_ethnicity-4,co-applicant_ethnicity-5,applicant_ethnicity_observed,co-applicant_ethnicity_observed,applicant_race-1,applicant_race-2,applicant_race-3,applicant_race-4,applicant_race-5,co-applicant_race-1,co-applicant_race-2,co-applicant_race-3,co-applicant_race-4,co-applicant_race-5,applicant_race_observed,co-applicant_race_observed,applicant_sex,co-applicant_sex,applicant_sex_observed,co-applicant_sex_observed,submission_of_application,initially_payable_to_institution,aus-1,aus-2,aus-3,aus-4,aus-5,denial_reason-1,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
count,377468.0,377468.000000,377468.000000,3.774680e+05,377468.000000,377468.000000,377468.000000,377468.000000,377468.000000,377468.000000,377468.0,377468.000000,377468.000000,3.774680e+05,3.750410e+05,204618.000000,195437.000000,377468.000000,82796.000000,0.0,82801.000000,34081.000000,45033.000000,377467.000000,219076.0,281122.000000,377468.000000,377468.000000,377468.000000,377468.0,3.750390e+05,377468.000000,377468.000000,377468.000000,377468.000000,93.000000,366330.000000,377468.000000,377468.000000,377092.000000,...,7870.000000,103.000000,5.000000,2.000000,377468.000000,377468.000000,377441.000000,30033.000000,1699.000000,123.000000,37.000000,377451.000000,12762.000000,659.000000,45.000000,13.000000,377468.000000,377468.000000,377468.000000,377468.000000,377468.000000,377468.000000,377468.000000,377468.000000,377468.000000,46723.000000,0.0,0.0,0.0,377468.000000,34201.000000,6172.000000,561.000000,377468.000000,377468.000000,377468.000000,377468.000000,377468.000000,377468.000000,377468.000000
mean,2018.0,33696.404583,23660.197553,2.366041e+10,1.947203,0.100472,1.987392,1.012515,10.667132,1.467584,2.0,1.312019,1.953909,2.311417e+05,8.298828e+01,4.290186,0.453760,2.490661,4600.166957,NaN,1640.022579,1619.523958,1614.154277,350.662098,36.0,7.786584,1.999995,1.955578,1.997369,2.0,5.937726e+05,1.000466,1.098308,2.999142,4.998283,28.311828,141.560050,6.697670,9.041810,2.130268,...,11.742313,12.970874,13.000000,13.500000,1.991517,3.111392,5.165191,20.541937,23.098882,26.471545,26.054054,6.762348,21.031186,21.959029,26.844444,20.769231,1.990881,3.111305,1.484187,3.562106,1.992892,3.111684,1.006159,1.000122,5.233946,4.952122,NaN,NaN,NaN,6.661693,4.831613,5.434867,7.327986,5728.295728,38.382259,80240.261161,125.710458,1438.530079,1902.086611,36.943291
std,0.0,16702.388628,16491.415584,1.649140e+10,0.982777,0.531139,0.111574,0.127632,13.116769,0.498949,0.0,0.463318,0.209683,8.534843e+05,3.820228e+03,0.907981,1.055004,0.499913,2641.559828,NaN,1890.870925,2686.975775,2231.184200,39.630907,0.0,24.379917,0.002302,0.206032,0.051223,0.0,3.254556e+06,0.021588,0.389522,0.041296,0.082592,43.249021,318.044444,2.700692,1.956759,1.245436,...,1.890489,1.175371,0.707107,0.707107,0.105995,0.998027,3.369449,7.368238,8.038020,13.032950,14.033583,2.578484,6.756316,8.607791,14.823132,12.590697,0.108902,0.998169,0.641547,1.663002,0.099406,0.997557,0.078240,0.015611,1.413125,0.375901,NaN,N

#  **Data Cleaning**

1.Dropping irrelevant columns

In [ ]:
drop_list=['hoepa_status','preapproval','lien_status','reverse_mortgage','open-end_line_of_credit','aus-1','activity_year','purchaser_type','lei','state_code','multifamily_affordable_units','census_tract',	'total_points_and_fees','co-applicant_ethnicity-2','co-applicant_ethnicity-3','co-applicant_ethnicity-4','co-applicant_ethnicity-5','applicant_race-2','applicant_race-3','applicant_race-4','applicant_race-5','co-applicant_race-2','co-applicant_race-3','co-applicant_race-4','co-applicant_race-5','aus-2','aus-3','aus-4','aus-5','denial_reason-2','denial_reason-3','denial_reason-4','applicant_ethnicity-2',
       'applicant_ethnicity-3', 'applicant_ethnicity-4','co-applicant_sex_observed','applicant_sex_observed','applicant_ethnicity_observed', 'co-applicant_ethnicity_observed','applicant_race_observed',
       'co-applicant_race_observed', 'applicant_ethnicity-5','prepayment_penalty_term','denial_reason-1','interest_only_payment','balloon_payment','negative_amortization','other_nonamortizing_features','interest_rate','total_loan_costs','origination_charges','discount_points','lender_credits','tract_median_age_of_housing_units','tract_one_to_four_family_homes','tract_owner_occupied_units','intro_rate_period','loan_to_value_ratio', 'manufactured_home_land_property_interest','manufactured_home_secured_property_type','construction_method','rate_spread','initially_payable_to_institution']


boa_df=boa_df.drop(drop_list, axis=1)


In [ ]:
len(drop_list)

63

2.Converting data type for categorical features

In [ ]:
cat_f=['submission_of_application', 'applicant_age_above_62','applicant_sex', 'co-applicant_sex',
 'applicant_ethnicity-1', 'co-applicant_ethnicity-1', 'co-applicant_race-1','applicant_credit_score_type', 'co-applicant_credit_score_type',
 'occupancy_type']
boa_df[cat_f]=boa_df[cat_f].astype(str)

In [ ]:
cf2=['loan_purpose','business_or_commercial_purpose']
boa_df[cf2]=boa_df[cf2].astype(str)

In [ ]:
boa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377468 entries, 0 to 377467
Data columns (total 36 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   derived_msa-md                     377468 non-null  int64  
 1   county_code                        377468 non-null  int64  
 2   conforming_loan_limit              377374 non-null  object 
 3   derived_loan_product_type          377468 non-null  object 
 4   derived_dwelling_category          377468 non-null  object 
 5   derived_ethnicity                  377468 non-null  object 
 6   derived_race                       377468 non-null  object 
 7   derived_sex                        377468 non-null  object 
 8   action_taken                       377468 non-null  int64  
 9   loan_type                          377468 non-null  int64  
 10  loan_purpose                       377468 non-null  object 
 11  business_or_commercial_purpose     3774

3.Missing Values Imputing

In [ ]:
# number of missing values in each columns
col_nan=boa_df.columns[boa_df.isnull().any()]
n_col_nan=len(boa_df.columns[boa_df.isnull().any()])
print(n_col_nan)
nan_sum=boa_df[col_nan].isna().sum()
nan_sum


7


conforming_loan_limit            94
loan_term                         1
property_value                 2429
income                        11138
debt_to_income_ratio           4808
applicant_race-1                 27
co-applicant_age_above_62    210312
dtype: int64

In [ ]:
#dropping unsignificant rows (since these are really little amount of data among 30K data we have)
boa_df=boa_df.dropna(subset=['conforming_loan_limit','loan_term','property_value','debt_to_income_ratio','applicant_race-1', 'co-applicant_race-1','co-applicant_race-1','applicant_ethnicity-1','co-applicant_ethnicity-1','applicant_age_above_62'])

In [ ]:
#dealing with missing values in co-applicant_age_above_62 column
boa_df['co-applicant_age_above_62'] = boa_df.apply(
    lambda row: 3 if pd.isnull(row['co-applicant_age_above_62']) else row['co-applicant_age_above_62'],
    axis=1)

In [ ]:
#creating a new column for missing income
boa_df['missing_income'] = boa_df.apply(lambda row: 1 if pd.isna(row['income']) else 0, axis=1)

In [ ]:
boa_df['missing_income']=boa_df['missing_income'].astype(str)

In [ ]:
col_corr=boa_df.corr(method='pearson')
col_corr['income'].sort_values(ascending=False,key=abs)

income                               1.000000
loan_amount                          0.463502
tract_to_msa_income_percentage       0.187248
property_value                       0.107028
action_taken                        -0.081417
ffiec_msa_md_median_family_income    0.073511
tract_minority_population_percent   -0.068951
loan_term                           -0.057398
county_code                         -0.022063
tract_population                    -0.021299
loan_type                           -0.019860
applicant_race-1                     0.008090
derived_msa-md                       0.007666
Name: income, dtype: float64

In [ ]:
# predicting missing income with loan amount since they are the most correlated 
available_income=boa_df.dropna(subset=['income'])
income_model = linear_model.LinearRegression().fit(available_income[['loan_amount']],available_income[['income']])
m_income=income_model.coef_
b_income=income_model.intercept_
boa_df['income'] = boa_df.apply(lambda row: m_income[0][0]*row['loan_amount']+b_income[0] if np.isnan(row['income']) else row['income'],axis=1)

In [ ]:
#approved1/denied0 column 
boa_df['action_taken'] = boa_df.apply(lambda row: 1 if row['action_taken']==1 or row['action_taken']==2 else 0,axis=1)

In [ ]:
col_corr['action_taken'].sort_values(ascending=False,key=abs)

action_taken                         1.000000
loan_amount                         -0.207202
tract_to_msa_income_percentage      -0.151305
tract_minority_population_percent    0.147943
loan_term                            0.130209
income                              -0.081417
ffiec_msa_md_median_family_income   -0.078459
property_value                      -0.021052
loan_type                           -0.013655
tract_population                     0.012710
applicant_race-1                     0.010029
derived_msa-md                      -0.001803
county_code                         -0.001036
Name: action_taken, dtype: float64

In [ ]:
boa_df=boa_df.drop(['derived_msa-md','county_code','tract_population','ffiec_msa_md_median_family_income', 'applicant_race-1','loan_type'], axis=1)


In [ ]:
boa_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371933 entries, 0 to 377467
Data columns (total 31 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   conforming_loan_limit              371933 non-null  object 
 1   derived_loan_product_type          371933 non-null  object 
 2   derived_dwelling_category          371933 non-null  object 
 3   derived_ethnicity                  371933 non-null  object 
 4   derived_race                       371933 non-null  object 
 5   derived_sex                        371933 non-null  object 
 6   action_taken                       371933 non-null  int64  
 7   loan_purpose                       371933 non-null  object 
 8   business_or_commercial_purpose     371933 non-null  object 
 9   loan_amount                        371933 non-null  float64
 10  loan_term                          371933 non-null  float64
 11  property_value                     3719

In [ ]:
boa_df.to_csv (path_or_buf=r'/drive/My Drive/orie4741 project/boa_data.csv')